In [1]:
import pandas as pd
import pyarrow.parquet as pq
import dask.dataframe as dd
from datetime import datetime
import os.path
from dotenv import load_dotenv
import s3fs
import awswrangler as wr

In [2]:
load_dotenv()
DATA_DIR = os.environ.get("DATA_DIR")

## Using pandas for querying by date

In [61]:
generator = wr.s3.read_parquet(path = "s3://jua-code-challenge-data/data/precipitation_data/", chunked=1_000_000)

In [56]:
pq_file_paths = []
for i in range(1,5):
    file_date = f'2022-05-{i:02d}'
    s3_url = f's3://jua-code-challenge-data/data/precipitation_data/precipitation_{file_date}.parquet.gzip'
    pq_file_paths.append(s3_url)

print(pq_file_paths)

['s3://jua-code-challenge-data/data/precipitation_data/precipitation_2022-05-01.parquet.gzip', 's3://jua-code-challenge-data/data/precipitation_data/precipitation_2022-05-02.parquet.gzip', 's3://jua-code-challenge-data/data/precipitation_data/precipitation_2022-05-03.parquet.gzip', 's3://jua-code-challenge-data/data/precipitation_data/precipitation_2022-05-04.parquet.gzip']


In [66]:
i = 0
for df in generator:
    i = i + 1
    df_time_index = df.reset_index(level=['lon','lat','nv'])
    pd.to_datetime(df_time_index.index)
    print(df_time_index.index.day)
    break
    

Int64Index([1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
            ...
            1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
           dtype='int64', name='time1', length=1000000)


In [5]:
s3 = s3fs.S3FileSystem()

pandas_dataframe = pq.ParquetDataset('s3://jua-code-challenge-data/data/precipitation_data/', filesystem=s3).read_pandas().to_pandas()

ArrowInvalid: Parquet file size is 0 bytes

In [42]:
df = pd.read_parquet('s3://jua-code-challenge-data/data/precipitation_data/', engine='pyarrow', chunksize=1000)    

TypeError: read_table() got an unexpected keyword argument 'chunksize'

In [ ]:
data_pd = pd.read_parquet(f"{DATA_DIR}\df.parquet.gzip")
print(data_pd.count()) # example of operation on the returned DataFrame

In [ ]:
data_pd

## Reset index

In [ ]:
data_pd.index

In [ ]:
start_time = pd.to_datetime("2022-05-01 21:00:00") #inclusive
end_time = pd.to_datetime("2022-05-01 23:00:00") #not inclusive

In [ ]:
# Reset index so the time1 is only index
data_pd_time_index = data_pd.reset_index(level=['lon','lat','nv'])

In [ ]:
# Set index to pandas datetime
data_pd_time_index.index = pd.to_datetime(data_pd_time_index.index)

In [ ]:
data_pd_time_index.index

In [ ]:
# Filter by time
data_pd_time_index.loc[(data_pd_time_index.index>=start_time)&(data_pd_time_index.index<end_time)]

## Filtering by the hierarchical geospatial index

In [ ]:
import h3

In [ ]:
data_pd_time_index

In [ ]:
data_pd_time_index['h3_index'] = data_pd_time_index.apply(lambda row: h3.geo_to_h3(row['lat'], row['lon'], 2), axis=1)

In [ ]:
data_pd_time_index

In [ ]:
h3_index = '820327fffffffff'
filtered_df = data_pd_time_index[data_pd_time_index['h3_index'] == h3_index]

In [ ]:
filtered_df.describe()

# Graveyard

## Plot data on map

In [ ]:
import geopandas
import matplotlib.pyplot as plt

In [ ]:
conda install geopandas

## Using pyarrow for querying by date

data_pa = pa.parquet.read_table(f"{DATA_DIR}df.parquet.gzip")

data_pa

timestamp_header = "time1"
#start_time = datetime('2022-04-30 21:00:00.00000')
#end_time = datetime('2022-05-01 18:00:00.000000')

date_string = "2022-04-30 21:00:00.000000"
date_object = pa.strptime(date_string, '%Y-%m-%d %H:%M:%S.%f')
start_time = pa.TimestampScalar(date_object, 'us')
print(start_time)

data_pq[timestamp_header][0]

data_pq[timestamp_header][0]

filtered_table = data_pq.filter((data_pq[timestamp_header] >= start_time) & (data_pq[timestamp_header] <= start_time)
)